# Etude d'un dataset pour prédire le prix d'une maison

- On va ici essayer de prédire le prix d'une maison en fonction de : 

- id : Identifiant de la vente,

- date : Date de la vente,

- nb_chambres : Nombre de chambres,

- nb_sdb : Nombre de salles de bain, les valeurs 0.5 correspondent aux WC sans douche/baignoire,

- m2_interieur : Surface en m2 de l'espace vivable de la maison,

- m2_jardin : Surface en m2 de l'espace extérieur,

- m2_etage : Surface en m2 des pièces situées au dessus du niveau du sol,

- m2_soussol : Surface en m2 des pièces situées en dessous du niveau du sol,

- nb_etages : Nombre d'étages,

- vue_mer : Variable booléenne indiquant s'il y a une vue sur mer ou non,

- vue_note : Note entre 0 et 4 indiquant la qualité de la vue,

- etat_note : Note de 1 à 5 indiquant l'état de la maison,

- design_note : Note de 1 à 13 indiquant le niveau de design de la maison,

- annee_construction : Année de construction,

- annee_renovation : Dernière année de rénovation s'il y a eu, sinon 0,

- m2_interieur_15voisins : Surface en m2 de l'espace vivable des 15 voisins les plus proches,

- m2_jardin_15voisins : Surface en m2 de l'espace extérieur des 15 voisins les plus proches,

- zipcode : Zipcode de l'adresse de la maison

- lat : latitude,

- long : longitude

In [1]:
import pandas as pd
import numpy as np


In [2]:
test_data = pd.read_csv("test_data.csv")
train_data = pd.read_csv("train_data.csv")
submission_sample = pd.read_csv("submission_sample.csv")

In [3]:
test_data.head(5)

,id,date,nb_chambres,nb_sdb,m2_interieur,m2_jardin,m2_etage,m2_soussol,nb_etages,vue_mer,vue_note,etat_note,design_note,annee_construction,annee_renovation,m2_interieur_15voisins,m2_jardin_15voisins,zipcode,lat,long
0,3600072,2015-03-30T00:00:00Z,4,2.75,206.243032,493.311037,108.695652,97.547380,1.0,0,0,5,7,1951,0,143.069491,390.189521,98144,47.5801,-122.294
1,6200017,2014-11-12T00:00:00Z,3,1.00,124.489038,1982.162765,124.489038,0.000000,1.5,0,0,4,5,1945,0,124.489038,3502.694166,98032,47.4023,-122.273
2,7600136,2014-07-18T00:00:00Z,2,2.00,104.979562,106.651802,74.321813,30.657748,2.0,0,0,3,9,2007,0,125.418060,111.575622,98122,47.6023,-122.314
3,11200400,2014-09-23T00:00:00Z,3,2.50,177.443330,416.945373,177.443330,0.000000,2.0,0,0,3,8,1998,0,142.140468,343.366778,98007,47.6176,-122.140
4,11500890,2015-03-12T00:00:00Z,3,2.50,290.784095,812.894835,290.784095,0.000000,2.0,0,0,3,10,1991,0,265.700483,836.399108,98052,47.6954,-122.103
